In [9]:
import numpy as np
import pandas as pd
import sklearn.model_selection as model_selection

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [10]:
# Đọc dữ liệu từ tập tin CSV
data = pd.read_csv('lottery.csv')

In [11]:
# Chuẩn bị dữ liệu
X = data.drop(columns=['Date']).values
y = data[['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine']].values

In [12]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Reshape lại dữ liệu để phù hợp với input của mạng LSTM
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [15]:
# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(10))  # 10 đầu ra tương ứng với số lần xuất hiện từ zero đến nine
model.compile(loss='mse', optimizer='adam')

In [17]:
# Huấn luyện mô hình
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=16, validation_data=(X_test_reshaped, y_test), verbose=2)

Epoch 1/100
350/350 - 0s - loss: 0.0094 - val_loss: 0.0104 - 385ms/epoch - 1ms/step
Epoch 2/100
350/350 - 0s - loss: 0.0094 - val_loss: 0.0099 - 363ms/epoch - 1ms/step
Epoch 3/100
350/350 - 0s - loss: 0.0091 - val_loss: 0.0102 - 364ms/epoch - 1ms/step
Epoch 4/100
350/350 - 0s - loss: 0.0090 - val_loss: 0.0097 - 356ms/epoch - 1ms/step
Epoch 5/100
350/350 - 0s - loss: 0.0089 - val_loss: 0.0092 - 371ms/epoch - 1ms/step
Epoch 6/100
350/350 - 0s - loss: 0.0086 - val_loss: 0.0098 - 358ms/epoch - 1ms/step
Epoch 7/100
350/350 - 0s - loss: 0.0086 - val_loss: 0.0091 - 363ms/epoch - 1ms/step
Epoch 8/100
350/350 - 0s - loss: 0.0083 - val_loss: 0.0092 - 367ms/epoch - 1ms/step
Epoch 9/100
350/350 - 0s - loss: 0.0083 - val_loss: 0.0089 - 368ms/epoch - 1ms/step
Epoch 10/100
350/350 - 0s - loss: 0.0080 - val_loss: 0.0089 - 367ms/epoch - 1ms/step
Epoch 11/100
350/350 - 0s - loss: 0.0079 - val_loss: 0.0088 - 361ms/epoch - 1ms/step
Epoch 12/100
350/350 - 0s - loss: 0.0079 - val_loss: 0.0088 - 370ms/epoch 

In [32]:
# Đánh giá mô hình trên tập kiểm tra
loss = model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f'Test Loss: {loss}')

Test Loss: 0.0030441652052104473


In [33]:
# Dự đoán số lần xuất hiện của các cột từ zero đến nine cho dữ liệu mới
new_data_scaled = scaler.transform(np.array([[10, 12, 12, 12, 10, 16, 12, 10, 6, 7]]))
predictions = model.predict(new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1])))
print(f'Predictions: {predictions}')

1/1 [==============================] - 0s 15ms/step
Predictions: [[ 9.982309  11.996092  12.024464  12.03833    9.977502  16.094246
  12.015838  10.043556   5.9246583  6.946287 ]]


In [34]:
# Tạo dictionary giữa các cột từ zero đến nine và dự đoán tương ứng
predictions_dict = {col_name: prediction for col_name, prediction in zip(['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'], predictions[0])}
print(f'Dictionary giữa dự đoán và các cột tương ứng: {predictions_dict}')

Dictionary giữa dự đoán và các cột tương ứng: {'Zero': 9.982309, 'One': 11.996092, 'Two': 12.024464, 'Three': 12.03833, 'Four': 9.977502, 'Five': 16.094246, 'Six': 12.015838, 'Seven': 10.043556, 'Eight': 5.9246583, 'Nine': 6.946287}


In [45]:
print(f'''Cặp số có giá trị dự đoán cao nhất: {dict(sorted(predictions_dict.items(), key=lambda x: x[1], reverse=True)[:2])}
Cặp số có giá trị dự đoán thấp nhất: {dict(sorted(predictions_dict.items(), key=lambda x: x[1])[:2])}''')

Cặp số có giá trị dự đoán cao nhất: {'Five': 16.094246, 'Three': 12.03833}
Cặp số có giá trị dự đoán thấp nhất: {'Eight': 5.9246583, 'Nine': 6.946287}
